# Get a copy of the repo

In [ ]:
!rm -rf examine_repo
!cp -r .backup examine_repo

In [ ]:
import git
import io
import json
from tqdm import tqdm
from glob import glob
import numpy as np

In [ ]:
repo = git.Repo('examine_repo')

In [ ]:
def iterate_jobs_content():
    for commit in tqdm(list(repo.iter_commits())):
        filename = commit.tree / 'jobs.json'
        with io.BytesIO(filename.data_stream.read()) as f:
            yield json.loads(f.read().decode('utf-8'))

In [ ]:
full = []

for jobs in iterate_jobs_content():
    if all([ c['status'] == 'completed' for c in jobs]):
        full.append(jobs)

In [ ]:
for i, j in enumerate(full):
    json.dump(j, open(f'jobs_run{i+1}.json', 'w'), indent=2)

In [ ]:
jsons = list(map(lambda x : json.load(open(x)), glob('jobs_run*.json')))

In [ ]:
agreements = np.asarray([ (all(c['has_object'] == 0 for c in j) or all(c['has_object'] == 1 for c in j)) for j in zip(*jsons) ])

In [ ]:
agreements.sum() / agreements.size

In [ ]:
result = [
    {
        'cluster': c0['cluster'],
        'has_object_0': c0['has_object'],
        'has_object_1': c1['has_object'],
        'has_object_2': c2['has_object'],
    }
    for c0, c1, c2 in zip(*jsons)
]

In [ ]:
json.dump(result, open('aggregate.json', 'w'), indent=2)